In [1]:
!pwd

/home/guest/Desktop/faceSwap2


In [ ]:
# Install Onnxruntime and Insightface

!sudo pip install onnxruntime
!sudo pip install insightface

In [3]:
# Set up git user name and email

!git config --global user.email "cheatedepk@gmail.com"
!git config --global user.name "epk2112"


In [4]:
import numpy as np
import os
import glob
import cv2
import matplotlib.pyplot as plt

import insightface
from insightface.app import FaceAnalysis
from insightface.data import get_image as ins_get_image


In [5]:
# Show the version of insightface and Numpy

print('insightface', insightface.__version__)
print('numpy', np.__version__)

insightface 0.7.3
numpy 1.22.4


In [ ]:
# This is the path to the folder where the images are stored